# Import libraries

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

# TOV solver functions
Modify functions of energy density `eden` function

Unit of pressure = unit of energy density = MeV/fm^3

In [ ]:
GS = 1.325e-12
mss = 1.1155e15

class tov:
    def __init__(self, ani, ras):
        self.ani = ani
        self.ras = ras
        self.fn = f"data_a={ani}_b={ras}.txt"

    def eden(self, P):
        conv = 7.5534e11
        a = 0.3
        b = 6.0 * 1e-20 * conv * conv
        eden = (P + math.sqrt(P**2 + 4*a*b))/(a*2)
        return eden

    def sig_pt(self, r, P, m):
        sigma = self.ani * P * 2 * GS * m * mss / (r * r)
        pt = P + sigma
        return sigma, pt

    def eden_eff(self, P):
        ras = self.ras
        ani = self.ani
        eden = self.eden(P)

        t = 3*P + 2*ani - eden

        eden_eff = eden + ras*t
        return eden_eff

    def P_eff(self, r, P, m):
        ras = self.ras
        ani = self.ani
        eden = self.eden(P)

        t = 3*P + 2*ani - eden
        sigma, Pt = self.sig_pt(r, P, m)

        Pr_eff = P - ras*t
        Pt_eff = Pt - ras*t
        return Pr_eff, Pt_eff

    #  dp/dr
    def dpdr(self, r, P, m, eden, Pr_eff, sigma):
        dpdr = - GS * eden * m * mss / (r*r) * (1 + P/eden)
        dpdr = dpdr * (1 + 4 * np.pi * r * r * r * Pr_eff / (m*mss))
        dpdr = dpdr / (1- 2 * GS*m*mss / r) - 2*sigma/r
        dsdp = self.ani * 2 * GS * m * mss / r
        dedp = derivative(self.eden, P)
        y = 1 - self.ras*(3 - 2*dsdp + dedp)
        dpdr = dpdr/y
        return dpdr*self.h

    #  dm/dr
    def dmdr(self, r, eden_eff):
        dmdr = 4.0 * np.pi * r*r * eden_eff
        return dmdr*self.h/mss

    #  dnu/dr
    def dnudr(self, r, m, P_eff):
        dnudr = GS*m*mss / (r * r) * (1 + 4*np.pi * r*r*r * P_eff/ (m*mss))
        dnudr = dnudr / (1 - 2 * GS*m*mss / r)
        return dnudr*self.h

    #  dω/dr
    def domgdr(self, r, P, m, nu, kapt, P_eff, eden, sigma):
        expnu = math.exp(nu)
        smgm = (1 - 2 * GS*m*mss / r)
        domgdr = 6.0 * expnu / math.sqrt(smgm) * kapt * (1 - sigma/(eden + P))
        domgdr = domgdr / (r**4)
        return domgdr*self.h

    #  dϰ/dr
    def dkapdr(self, r, press, m, nu, omgt, eden):
        expmnu = math.exp(-nu)
        smgm = (1 - 2 * GS*m*mss / r)
        dkapdr = GS * 8 * np.pi * (r**4) * expmnu * eden / 3 * (1 + press/eden)
        dkapdr = dkapdr * omgt / math.sqrt(smgm)
        return dkapdr*self.h

    #  dy/dr (y = ytidal)
    def dydr(self, r, press, m, y, eden, E_eff, P_eff):
        G = GS

        a = self.ani*2*G*m*mss/r + 1
        smgm =(1 - 2*G*m*mss/r)
        srp = (1 + 4*np.pi*r*r*r*press/(m*mss))
        dqyt = srp*srp/(smgm*smgm)

        f_tidal = (1 - 4*np.pi*r*r*G*(E_eff - P_eff)) / smgm

        # dsdp = self.ani * 2 * GS * m * mss / r
        # dedp = derivative(self.eden, press)

        dedp_eff = derivative(self.eden_eff, press)

        suku = 4*E_eff + 8*P_eff + ((dedp_eff+1)*(eden+press)/(a))

        q_tidal = 4 * np.pi * r * r * G * suku
        q_tidal = (q_tidal-6)/smgm
        q_tidal = q_tidal - 4*(G*m*mss)*(G*m*mss)*dqyt/(r*r)

        dydr = -(y*y + y*f_tidal + q_tidal)/r
        return dydr*self.h

    #  compute k2 and Lambda (tidal parameter)
    def compute_k2_lambda(self,y,m,r):
        G = GS
        # y = y(r)
        C = G*m*mss/r

        f1 = (4*(y+1)*C**4 +(6*y-4)*C**3+(26-22*y)*C**2+ 3*(5*y-8)*C -3*y+6)*2*C
        f2 = -3*(1-2*C)*(1-2*C)*(2*C*(y-1)-y+2)*np.log(1/(1-2*C))

        f = f1 + f2

        k2 = (8/5)*(1-2*C)*(1-2*C)*C**5*(2*C*(y-1)-y+2)/f
        lmbd = 2*k2 / (3 * C**5)
        lambda1 = 2*k2*(r**5)*1e-36/(3*6.674e-8)
        return k2, lmbd, lambda1


    #  RungeKutta to compute initial conditon
    def funct(self, ya, xa):
          G = GS
          h = self.h
          P = ya[0]
          m = ya[1]
          nu = ya[2]
          eden = ya[3]
          sigma = ya[4]

          r = xa

          P_eff, Pt_eff = self.P_eff(r, P, m)
          eden_eff = self.eden_eff(P)

          ek = [0] * 3
          ek[0] = self.dpdr(r, P, m, eden, P_eff, sigma)
          ek[1] = self.dmdr(r, eden_eff)
          ek[2] = self.dnudr(r, m, P_eff)
          return ek


    #  RungeKutta to compute TOV, momin, ytidal
    def funcx(self, ya, xa):
          G = GS
          h = self.h
          P = ya[0]
          m = ya[1]
          nu = ya[2]
          omgt = ya[3]
          kapt = ya[4]
          y = ya[5]
          eden = ya[6]

          r =  xa

          sigma, pt = self.sig_pt(r, P, m)
          P_eff, Pt_eff = self.P_eff(r, P, m)
          eden_eff = self.eden_eff(P)

          ek = [0] * 6
          ek[0] = self.dpdr(r, P, m, eden, P_eff, sigma)
          ek[1] = self.dmdr(r, eden_eff)
          ek[2] = self.dnudr(r, m, P_eff)
          ek[3] = self.domgdr(r, P, m, nu, kapt, P_eff, eden, sigma)
          ek[4] = self.dkapdr(r, P, m, nu, omgt, eden)
          ek[5] = self.dydr(r, P, m, y, eden, eden_eff, P_eff)
          return ek


    #  compute initial conditon matric nu
    def tovi(self, pcc, tuct):
      eden = self.eden(pcc)
      sigma, pt = self.sig_pt(1e-3, pcc, 1e-9)
      y = [0] * 6
      # y: 0-press, 1-m, 2-nu, 3-eden, 4-rho
      y[0] = pcc
      y[1] = 1e-9
      y[2] = tuct
      y[3] = eden
      y[4] = sigma
      y[5] = pt

      r_interval = 0.1    # interval of radius for printing
      steps = 16          # number of steps in one print
      r_max = 20e3        # max radius to stop calculating

      h = r_interval/steps
      self.h = h
      xp = 1e-3
      li = 0
      ps = pcc
      ya = [0] * 6
      ek = [[0 for j in range(3)] for i in range(4)]

      while (ps > 1e-10):
            li = li + 1
            for i in range(steps):
                xb = xp
                xp = xp + h
                xm = xb + h/2

                # RK-1
                for k in range (6):
                  ya[k] = y[k]
                xa = xb

                ekj = self.funct(ya, xa)
                for k in range (3):
                 ek[0][k] = ekj[k]

                # RK-2
                for k in range (3):
                  ya[k] = y[k] + (ek[0][k])/2
                xa = xm
                eden = self.eden(ya[0])
                sigma, pt = self.sig_pt(xa, ya[0], ya[1])
                ya[3] = eden
                ya[4] = sigma
                ya[5] = pt

                ekj = self.funct(ya, xa)

                for k in range (3):
                 ek[1][k] = ekj[k]

                # RK-3
                for k in range (3):
                  ya[k] = y[k] + (ek[1][k])/2
                xa = xm
                eden = self.eden(ya[0])
                sigma, pt = self.sig_pt(xa, ya[0], ya[1])
                ya[3] = eden
                ya[4] = sigma
                ya[5] = pt

                ekj = self.funct(ya, xa)
                for k in range (3):
                 ek[2][k] = ekj[k]

                # RK-4
                for k in range (3):
                  ya[k] = y[k] + (ek[2][k])
                xa = xp
                eden = self.eden(ya[0])
                sigma, pt = self.sig_pt(xa, ya[0], ya[1])
                ya[3] = eden
                ya[4] = sigma
                ya[5] = pt

                ekj = self.funct(ya, xa)
                for k in range (3):
                 ek[3][k] = ekj[k]

                for k in range (3):
                 y[k] = y[k] + (ek[0][k] + 2*ek[1][k] + 2*ek[2][k] + ek[3][k])/6
                eden = self.eden(y[0])
                sigma, pt = self.sig_pt(xa, y[0], y[1])
                y[3] = eden
                y[4] = sigma
                y[5] = pt

            ps = y[0]
      r0 = xp/1000
      m0 = y[1]
      nu0 = y[2]
      return ps, m0, nu0, r0, eden, sigma, pt

    #  solve 1st order differential functions
    def tovmi(self, pcc, tuct):
          eden = self.eden(pcc)
          sigma, pt = self.sig_pt(1e-3, pcc, 1e-9)
          y = [0] * 7
          # urutan y: 1-press, 2-m, 3-nu, 4-omet, 5-kapat, 6-y, 7-eden, 8-rho
          y[0] = pcc
          y[1] = 1e-9
          y[2] = tuct
          y[3] = 1e-9
          y[4] = 1e-9
          y[5] = 2
          y[6] = eden

          r_interval = 0.1    # interval of radius for printing
          steps = 32          # number of steps in one print
          r_max = 30e3        # max radius to stop calculating

          h = r_interval/steps
          self.h = h
          xp = 1e-3
          li = 0
          ps = pcc

          ya = [0] * 7
          ek = [[0 for j in range(6)] for i in range(4)]

          while (ps > 1e-10):
            li = li + 1

            for i in range(steps):
              xb = xp
              xp = xp + h
              xm = xb + h/2

              # RK-1
              j = 0
              for k in range (7):
                ya[k] = y[k]
              xa = xb

              ekj = self.funcx(ya, xa)
              for k in range (6):
                ek[j][k] = ekj[k]

              # RK-2
              j = 1
              for k in range (6):
                ya[k] = y[k] + (ek[0][k])/2
              xa = xm
              eden = self.eden(ya[0])
              ya[6] = eden

              ekj = self.funcx(ya, xa)
              for k in range (6):
                ek[j][k] = ekj[k]


              # RK-3
              j = 2
              for k in range (6):
                ya[k] = y[k] + (ek[1][k])/2
              xa = xm
              eden = self.eden(ya[0])
              ya[6] = eden

              ekj = self.funcx(ya, xa)
              for k in range (6):
                ek[j][k] = ekj[k]

              # RK-4
              j = 3
              for k in range (6):
                ya[k] = y[k] + (ek[2][k])
              xa = xp
              eden = self.eden(ya[0])
              ya[6] = eden

              ekj = self.funcx(ya, xa)
              for k in range (6):
                ek[j][k] = ekj[k]

              for k in range (6):
                y[k] = y[k] + (ek[0][k] + 2*ek[1][k] + 2*ek[2][k] + ek[3][k])/6
              eden = self.eden(y[0])
              y[6] = eden

            ps = y[0]


          r0 = xp/1000
          m0 = y[1]
          nu0 = y[2]
          omgt0 = y[3]
          kapt0 = y[4]
          y0 = y[5]
          sigma, pt = self.sig_pt(r0, pcc, m0)
          return ps, m0, nu0, r0, omgt0, kapt0, y0, eden, sigma, pt

    # get moment of inertia
    def get_momin(self,G,m,r,omgt0,kapt0):
      rns = r*1000
      eta = 1/(omgt0 + 2*kapt0/(r*r*r))
      omega = eta*omgt0
      kappa = eta*kapt0
      momin = kappa/(G*m*mss*rns*rns)             # I/MR^2 dimensionlesss
      m_i = momin*1.98892e33*(m*mss)*r*r/1e35     # moment of inertia in 10^45 g cm^2
      return momin, m_i

    def get_vs(self, pr, pt):
      dedpr = derivative(self.eden, pr)
      dedpt = derivative(self.eden, pt)
      dedpr_eff = derivative(self.eden_eff, pr)
      dedpt_eff = derivative(self.eden_eff, pt)

      v2r = 1/dedpr
      v2t = 1/dedpt
      v2r_eff = 1/dedpr_eff
      v2t_eff = 1/dedpt_eff
      return v2r, v2t, v2r_eff, v2t_eff

    def get_ec(self, eden, pr, pt):
      # eden, pr, pt yang masuk harus sudah effective
      ec1 = eden - pr - 2*pt
      ec2 = eden + pr + 2*pt
      return ec1, ec2


    # main function
    def solve(self):
      arr = []
      fn = self.fn
      print("Results will be saved to file: ", fn)
      for i in range(1, 500, 2):
        pcc = i
        tuct = 1e-9
        G = GS

        p, m, mnurt, r, eden, sigma, pt = self.tovi(pcc, tuct)

        rns = r*1000
        dbs = 1-2*G*m*mss/rns
        kc = 0.5 * np.log(dbs) - mnurt
        rot = tuct + kc

        P0, m, nu0, r, omgt0, kapt0, y0, eden, sigma, pt = self.tovmi(pcc, rot)


        eden = self.eden(pcc)
        eden_eff = self.eden_eff(pcc)
        sigma, pt = self.sig_pt(r, pcc, m)
        Pr_eff, Pt_eff = self.P_eff(r, pcc, m)

        momin, mi = self.get_momin(G,m,r,omgt0,kapt0)
        k2, lmbd, lambda1 = self.compute_k2_lambda(y0,m,r*1000)
        v2r, v2t, v2r_eff, v2t_eff = self.get_vs(pcc, pt)
        ec1, ec2 = self.get_ec(eden_eff, Pr_eff, Pt_eff)

        comp = m/r                 #compactness
        log_eden = np.log10(eden)

        smgm = 1 - 2*m*1.48/r
        zsur = 1/math.sqrt(smgm) - 1

        arr.append([pcc, r, m, eden, log_eden, comp, momin, mi, k2, lmbd,
                    lambda1, sigma, pt, eden_eff, Pr_eff, Pt_eff, v2r, v2t,
                    v2r_eff, v2t_eff, ec1, ec2, zsur])
        print(pcc, r, m)
        np.savetxt(fn, arr)

      return arr


def derivative(f,x,h=0.01):
    return (f(x-2*h)-8*f(x-h)+8*f(x+h)-f(x+2*h))/(12*h)



# Run main code
*   input anisotropy parameter alpha `ani` and Rastall parameter beta `ras`

*   Data saved to `.txt` file
*   Printed outputs are radial pressure (MeV/fm3), radius (km), and mass (solar mass)

In [ ]:
ani = 0     # set 0 for isotropes
ras = 0     # set 0 for GR
ds = tov(ani, ras)

arr = ds.solve()